# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [15]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [16]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob
import pandas as pd

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)

In [25]:
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [26]:
# Write your code below.
dd_feat = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1),
                               returns = (x['Close']/x['Close'].shift(1)) - 1,
                               adjClose_lag_1 = x['Adj Close'].shift(1),
                               hi_low_range = x.High - x.Low), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8', #int64
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'returns': 'f8',
                    'adjClose_lag_1': 'f8',
                    'hi_low_range': 'f8'},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))


In [19]:
df = dd_feat.compute() #results are as sometimes expected. But ocassionally running the same chunk will give different results, and recreating the dd_feat object by running chunk above does not resolve
df.head() 


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,returns,adjClose_lag_1,hi_low_range
ticker,,,,,,,,,,,,,
ACHC,1994-03-04,24.0,24.000,22.875,24.000,24.000,17900.0,ACHC.csv,1994,NaN,NaN,NaN,0.26
ACHC,1994-03-07,23.5,25.375,23.500,25.375,25.375,68100.0,ACHC.csv,1994,9.92,-0.018145,9.92,0.34
ACHC,1994-03-08,24.5,25.250,24.000,24.750,24.750,9200.0,ACHC.csv,1994,9.74,0.003080,9.74,0.57
ACHC,1994-03-09,24.0,24.000,24.000,24.000,24.000,100.0,ACHC.csv,1994,9.77,0.162743,9.77,1.81
ACHC,1994-03-10,24.0,24.500,24.000,24.500,24.500,14300.0,ACHC.csv,1994,11.36,-0.005282,11.36,0.48


In [ ]:
#Compare Hi-Low and Close_lag calculated as DD df vs PD df:
df = df.assign(hi_low_2 = df["High"] - df["Low"])

df = df.assign(Close_lag_1_pd_DF = lambda x: x['Close'].shift(1))

df.head() #results match expected (ie first hi_low_2 is 1.125 vs 0.26)


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,returns,adjClose_lag_1,hi_low_range,hi_low_2,Close_lag_1_pd_DF
ticker,,,,,,,,,,,,,,,
ACHC,1994-03-04,24.0,24.000,22.875,24.000,24.000,17900.0,ACHC.csv,1994,NaN,NaN,NaN,0.26,1.125,NaN
ACHC,1994-03-07,23.5,25.375,23.500,25.375,25.375,68100.0,ACHC.csv,1994,9.92,-0.018145,9.92,0.34,1.875,24.000
ACHC,1994-03-08,24.5,25.250,24.000,24.750,24.750,9200.0,ACHC.csv,1994,9.74,0.003080,9.74,0.57,1.250,25.375
ACHC,1994-03-09,24.0,24.000,24.000,24.000,24.000,100.0,ACHC.csv,1994,9.77,0.162743,9.77,1.81,0.000,24.750
ACHC,1994-03-10,24.0,24.500,24.000,24.500,24.500,14300.0,ACHC.csv,1994,11.36,-0.005282,11.36,0.48,0.500,24.000


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [29]:
# Convert Dask dataframe to pandas dataframe
df_pd = dd_feat.compute() #compute might give aberrant results

#calculate 10-window moving average of returns
df_pd['MAvgReturns'] = df_pd.groupby('ticker')['returns'].transform(lambda x: x.rolling(10).mean())

df_pd.head(20) #use 20 for head since first 10 rows are NaN for moving average with a window of 10




,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,returns,adjClose_lag_1,hi_low_range,MAvgReturns
ticker,,,,,,,,,,,,,,
ACHC,1994-03-04,24.00,24.000,22.875,24.000,24.000,17900.0,ACHC.csv,1994,NaN,NaN,NaN,0.260000,NaN
ACHC,1994-03-07,23.50,25.375,23.500,25.375,25.375,68100.0,ACHC.csv,1994,9.92,-0.018145,9.92,0.340000,NaN
ACHC,1994-03-08,24.50,25.250,24.000,24.750,24.750,9200.0,ACHC.csv,1994,9.74,0.003080,9.74,0.570000,NaN
ACHC,1994-03-09,24.00,24.000,24.000,24.000,24.000,100.0,ACHC.csv,1994,9.77,0.162743,9.77,1.810000,NaN
ACHC,1994-03-10,24.00,24.500,24.000,24.500,24.500,14300.0,ACHC.csv,1994,11.36,-0.005282,11.36,0.480000,NaN
ACHC,1994-03-11,24.00,24.500,24.000,24.500,24.500,1300.0,ACHC.csv,1994,11.30,0.014159,11.30,0.760000,NaN
ACHC,1994-03-14,24.50,24.500,24.500,24.500,24.500,0.0,ACHC.csv,1994,11.46,-0.012216,11.46,0.290000,NaN
ACHC,1994-03-15,24.00,24.000,24.000,24.000,24.000,0.0,ACHC.csv,1994,11.32,0.007067,11.32,0.440000,NaN
ACHC,1994-03-16,24.00,24.125,24.000,24.125,24.125,3600.0,ACHC.csv,1994,11.40,-0.007018,11.40,0.270000,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

#I think the moving average could be calculated in Dask, even if the grouping variable spanned partitions. I would expect it to be faster to run the calculation in Dask if it could perform the moving average calculations by ticker in parallel, and it would be necessary if the dataframe was too big to fit in memory.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.